In [1]:
import cfbd
import numpy as np
import pandas as pd
import pickle
from sklearn import tree
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier

from sklearn.neighbors import KNeighborsClassifier as knn


In [15]:
!pip freeze

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


absl-py==1.2.0
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
astunparse==1.6.3
attrs==22.1.0
backcall==0.2.0
beautifulsoup4==4.11.1
bleach==5.0.1
blis==0.7.8
cachetools==5.2.0
catalogue==2.0.8
certifi==2022.5.18.1
cfbd==4.4.2
cffi==1.15.1
charset-normalizer==2.0.12
click==8.1.3
colorama==0.4.4
cycler==0.11.0
cymem==2.0.6
debugpy==1.6.0
decorator==5.1.1
defusedxml==0.7.1
entrypoints==0.4
executing==0.8.3
fastai==2.7.9
fastcore==1.5.24
fastdownload==0.0.7
fastjsonschema==2.16.1
fastprogress==1.0.3
flatbuffers==2.0.7
fonttools==4.37.1
gast==0.4.0
google-auth==2.11.1
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.48.1
h5py==3.7.0
idna==3.3
importlib-metadata==4.12.0
importlib-resources==5.9.0
ipykernel==6.13.0
ipython==8.3.0
ipython-genutils==0.2.0
jedi==0.18.1
Jinja2==3.1.2
joblib==1.1.0
jsonschema==4.15.0
jupyter-client==7.3.1
jupyter-core==4.10.0
jupyterlab-pygments==0.2.2
keras==2.10.0
Keras-Preprocessing==1.1.2
kiwisolver==1.4.4
langcodes==3.3.0
libclang

In [2]:
df = pd.read_csv('../cfb_imputed.csv')
test = pd.read_csv('../a_CFB_simple/df22.csv')
week4 = pd.read_csv('../a_CFB_simple/c.csv')
predictionsDB = pd.read_csv('../a_CFB_simple/d.csv')

In [3]:
df = df.drop('Unnamed: 0',axis=1)
df.head()

,id,year,week,neutral_site,home_team,home_conference,home_points,home_elo,away_team,away_conference,...,d_explosiveness,d_successrate,d_stuffrate,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread,margin
0,400763593,2015,1,False,UCF,American Athletic,14,1626,Florida International,Conference USA,...,1.397643,0.362319,0.233333,15.894630,1.365478,0.406250,0.466667,4.270422,-17.0,1
1,400603840,2015,1,True,South Carolina,SEC,17,1646,North Carolina,ACC,...,1.115637,0.461538,0.142857,7.968428,1.130578,0.426667,0.170213,9.157397,-3.5,-4
2,400763399,2015,1,False,Central Michigan,Mid-American,13,1417,Oklahoma State,Big 12,...,1.234668,0.426471,0.181818,18.608247,1.022896,0.416667,0.333333,8.286312,20.5,11
3,400603839,2015,1,False,Vanderbilt,SEC,12,1365,Western Kentucky,Conference USA,...,1.001898,0.396226,0.380952,4.754122,1.041876,0.397436,0.292683,-3.685131,-17.5,2
4,400756883,2015,1,False,Utah,Pac-12,24,1603,Michigan,Big Ten,...,1.163096,0.452055,0.241379,-1.184796,1.085531,0.394366,0.250000,7.254696,-3.0,-7


In [4]:
y = []
for i in range(len(df)):
    if df.spread[i] < 0:
        if df.margin[i] > df.spread[i]:
            y.append(0)
        else:
            y.append(1)
    else:
        if df.margin[i] < df.spread[i]:
            y.append(1)
        else:
            y.append(0)
            
df["home_team_cover_spread"] = y
df.head()

,id,year,week,neutral_site,home_team,home_conference,home_points,home_elo,away_team,away_conference,...,d_successrate,d_stuffrate,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread,margin,home_team_cover_spread
0,400763593,2015,1,False,UCF,American Athletic,14,1626,Florida International,Conference USA,...,0.362319,0.233333,15.894630,1.365478,0.406250,0.466667,4.270422,-17.0,1,0
1,400603840,2015,1,True,South Carolina,SEC,17,1646,North Carolina,ACC,...,0.461538,0.142857,7.968428,1.130578,0.426667,0.170213,9.157397,-3.5,-4,1
2,400763399,2015,1,False,Central Michigan,Mid-American,13,1417,Oklahoma State,Big 12,...,0.426471,0.181818,18.608247,1.022896,0.416667,0.333333,8.286312,20.5,11,1
3,400603839,2015,1,False,Vanderbilt,SEC,12,1365,Western Kentucky,Conference USA,...,0.396226,0.380952,4.754122,1.041876,0.397436,0.292683,-3.685131,-17.5,2,0
4,400756883,2015,1,False,Utah,Pac-12,24,1603,Michigan,Big Ten,...,0.452055,0.241379,-1.184796,1.085531,0.394366,0.250000,7.254696,-3.0,-7,1


In [5]:
excluded = ['id','year','week','home_team','away_team','margin', 'home_points', 'away_points']
cat_features = ['home_conference','away_conference','neutral_site']
cont_features = [c for c in df.columns.to_list() if c not in cat_features and c not in excluded]

In [6]:
cats = df[cat_features]
cats = cats.apply(preprocessing.LabelEncoder().fit_transform)
cats

,home_conference,away_conference,neutral_site
0,1,4,0
1,9,0,1
2,6,2,0
3,9,4,0
4,8,3,0
...,...,...,...
4739,9,9,1
4740,0,0,1
4741,3,3,1
4742,8,8,0


In [7]:
df = df.drop(['neutral_site', 'home_conference', 'away_conference','id','week','home_team','away_team','margin', 'home_points', 'away_points'],axis=1)
df = df.join(cats)
df = df.drop('attendance', axis=1)
df.head()

,year,home_elo,away_elo,home_sp_plus,home_second_order_w,home_sos,home_special_teams_rating,home_d_r_explosivenenss,home_d_r_success,home_d_havoc_db,...,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread,home_team_cover_spread,home_conference,away_conference,neutral_site
0,2015,1626,1321,-15.5,1.0,0.925,1.8,92.7,90.3,0.056,...,15.894630,1.365478,0.406250,0.466667,4.270422,-17.0,0,1,4,0
1,2015,1646,1477,3.4,4.1,0.820,0.8,105.5,87.6,0.039,...,7.968428,1.130578,0.426667,0.170213,9.157397,-3.5,1,9,0,1
2,2015,1417,1567,-3.1,7.2,0.935,-2.3,102.7,98.6,0.064,...,18.608247,1.022896,0.416667,0.333333,8.286312,20.5,1,6,2,0
3,2015,1365,1521,-1.6,4.1,0.842,-0.1,98.3,112.2,0.085,...,4.754122,1.041876,0.397436,0.292683,-3.685131,-17.5,0,9,4,0
4,2015,1603,1553,11.0,7.0,0.858,2.0,109.1,109.2,0.072,...,-1.184796,1.085531,0.394366,0.250000,7.254696,-3.0,1,8,3,0


In [8]:
test_df = df.query("year == 2021")
train_df = df.query("year != 2021")
train_df.head()

,year,home_elo,away_elo,home_sp_plus,home_second_order_w,home_sos,home_special_teams_rating,home_d_r_explosivenenss,home_d_r_success,home_d_havoc_db,...,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread,home_team_cover_spread,home_conference,away_conference,neutral_site
0,2015,1626,1321,-15.5,1.0,0.925,1.8,92.7,90.3,0.056,...,15.894630,1.365478,0.406250,0.466667,4.270422,-17.0,0,1,4,0
1,2015,1646,1477,3.4,4.1,0.820,0.8,105.5,87.6,0.039,...,7.968428,1.130578,0.426667,0.170213,9.157397,-3.5,1,9,0,1
2,2015,1417,1567,-3.1,7.2,0.935,-2.3,102.7,98.6,0.064,...,18.608247,1.022896,0.416667,0.333333,8.286312,20.5,1,6,2,0
3,2015,1365,1521,-1.6,4.1,0.842,-0.1,98.3,112.2,0.085,...,4.754122,1.041876,0.397436,0.292683,-3.685131,-17.5,0,9,4,0
4,2015,1603,1553,11.0,7.0,0.858,2.0,109.1,109.2,0.072,...,-1.184796,1.085531,0.394366,0.250000,7.254696,-3.0,1,8,3,0


In [9]:
trainx = train_df.drop(['home_team_cover_spread', 'year'], axis=1)
testx = test_df.drop(['home_team_cover_spread', 'year'], axis=1)

In [10]:
trainy = train_df[["home_team_cover_spread"]]
testy = test_df[["home_team_cover_spread"]]

In [11]:
model = knn(n_neighbors=3)

In [12]:
model.fit(trainx, trainy)

C:\Users\magamp\AppData\Roaming\Python\Python38\site-packages\sklearn\neighbors\_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [13]:
preds = model.predict(testx)

In [14]:
confusion_matrix(testy,preds)

array([[187, 170],
       [175, 198]], dtype=int64)